In [1]:
import pandas as pd

data_path = "metabolic_syndrome.csv"
df = pd.read_csv(data_path)
df


,seqn,age,sex,marital,income,race,waistcirc,bmi,albuminuria,uralbcr,uricacid,bloodglucose,hdl,triglycerides,metabolicsyndrome
0,62161,22,Male,Single,8200.0,White,81.0,23.3,0,3.88,4.9,92,41,84,No MetSyn
1,62164,44,Female,Married,4500.0,White,80.1,23.2,0,8.55,4.5,82,28,56,No MetSyn
2,62169,21,Male,Single,800.0,Asian,69.6,20.1,0,5.07,5.4,107,43,78,No MetSyn
3,62172,43,Female,Single,2000.0,Black,120.4,33.3,0,5.22,5.0,104,73,141,No MetSyn
4,62177,51,Male,Married,NaN,Asian,81.1,20.1,0,8.13,5.0,95,43,126,No MetSyn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2396,71901,48,Female,Married,1000.0,Other,NaN,59.7,0,22.11,5.8,152,57,107,No MetSyn
2397,71904,30,Female,Single,2000.0,Asian,NaN,18.0,0,2.90,7.9,91,90,91,No MetSyn
2398,71909,28,Male,Single,800.0,MexAmerican,100.8,29.4,0,2.78,6.2,99,47,84,No MetSyn
2399,71911,27,Male,Married,8200.0,MexAmerican,106.6,31.3,0,4.15,6.2,100,41,124,MetSyn


In [2]:
from statsmodels.formula.api import ols
import statsmodels.api as sm
import pandas as pd


df = pd.read_csv("metabolic_syndrome.csv")


numeric_vars = df.select_dtypes(include='number').columns.drop('seqn')


df_male = df[df['sex'] == 'Male']
df_female = df[df['sex'] == 'Female']


def perform_detailed_anova(df, var, group_col='metabolicsyndrome'):
    model = ols(f'{var} ~ C({group_col})', data=df).fit()
    anova_results = sm.stats.anova_lm(model, typ=2)
    p_value = anova_results['PR(>F)'][0]
    significant = "Yes" if p_value < 0.05 else "No"
    return {
        'Variable': var,
        'Sum of Squares': anova_results['sum_sq'][0],
        'Degrees of Freedom': anova_results['df'][0],
        'F-Value': anova_results['F'][0],
        'p-value': p_value,
        'Significant': significant
    }


def gather_anova_results(df, numeric_vars, sex):
    results = [perform_detailed_anova(df, var) for var in numeric_vars]
    anova_df = pd.DataFrame(results)
    anova_df['Sex'] = sex  


detailed_anova_results_male = gather_anova_results(df_male, numeric_vars, 'Male')
detailed_anova_results_female = gather_anova_results(df_female, numeric_vars, 'Female')



In [3]:
detailed_anova_results_female